In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import re
import time
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
import urllib
from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup
import http.cookiejar

meijer = 'https://www.meijer.com/shop/en/produce/fruit/c/L3-3197'
ins_aldi = "https://www.instacart.com/store/aldi/departments/282/aisles/57052"
walmart  = 'https://www.walmart.com/cp/food/976759'
ins_sch = 'https://www.instacart.com/store/schnucks/departments/8109/aisles/42777'

headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.183 Safari/537.36",
           "Accept-Encoding":"gzip, deflate, br"}

## Meijer

In [2]:
r = requests.get(meijer,headers=headers)
content = r.content
soup = BeautifulSoup(content,"html.parser",from_encoding='utf8')

fruits = {'store':[] ,'name':[],'price':[],'unit':[],'image_url':[]}  #'availability':[],
fruits['store'] = 'Meijer1'
for p in soup.find_all('div',attrs={'class':'product-tile-container'}):
    # extract name
    pn = p.find('a',class_='h7')
    if pn is None:
        continue
    else:
        fruits['name'].append(pn.text.strip().split(',')[0])
    
    # extract price 
    pp = p.find('div',class_='display-price')
    if pp is None:
        continue
    else:
        pp_list = pp.text.strip().split()
        if len(pp_list) == 2: #[latest_price, unit]
            uprice = float(pp_list[0][1:])
            unit = pp_list[1]
        if len(pp_list) == 3: #[latest_price, old_price, unit]
            uprice = float(pp_list[0][1:])
            unit = pp_list[2]
        if len(pp_list) == 4: #sale format:[Buy,num,for,total_cost]
            uprice = float(pp_list[3][1:])/float(pp_list[1])
            unit = 'Each'
    fruits['price'].append(uprice)
    fruits['unit'].append(unit)
    
    # extract image url
    img = p.find('div',class_='tile-row details-container')
    if img is None:
        continue
    else:
        for i in img.find_all('img',alt=True):
            fruits['image_url'].append(i['src'])
        
meijer_df = pd.DataFrame(fruits)
meijer_df.head()

,store,name,price,unit,image_url
0,Meijer1,Banana,0.24,Each,https://www.meijer.com/content/dam/meijer/prod...
1,Meijer1,Strawberries,2.50,Each,https://www.meijer.com/content/dam/meijer/prod...
2,Meijer1,Raspberries,1.00,Each,https://www.meijer.com/content/dam/meijer/prod...
3,Meijer1,Avocado,1.00,Each,https://www.meijer.com/content/dam/meijer/prod...
4,Meijer1,Blueberries,2.50,Each,https://www.meijer.com/content/dam/meijer/prod...


### JSON file (InstaCart)

In [4]:
import json
root = '/'
aldi_file1 = root + 'instacart_part1.json'
aldi_file2 = root + 'instacart_part2.json'
aldi_all = [aldi_file1,aldi_file2]
gfs_file = root + 'GFS_part1.json'
gfs_all = [gfs_file]
sch_part1 = root + 'sch_part1.json'
sch_part2 = root + 'sch_part2.json'
sch_part3 = root + 'sch_part3.json'
sch_part4 = root + 'sch_part4.json'
sch_all = [sch_part1,sch_part2,sch_part3,sch_part4]
target_file = root + 'target.json'
target_all = [target_file]

In [5]:
def load_json(file_path,store):
    with open(file_path) as f:
        json_data = json.load(f)
    #print(json.dumps(json_data,indent=4,sort_keys=True))
    items = json_data['module_data']['items']
    
    instart_fruits = {'name':[],'store':[],'price':[],'unit':[],'image_url':[]}
    instart_fruits['store'] = store
    for item in items:
        #instart_fruits['id'].append(item['id'])
        instart_fruits['image_url'].append(item['image_list'][0]['url'])
        n_list = item['name'].split(',')
        instart_fruits['name'].append(n_list[0])
        if len(n_list) > 1:
            instart_fruits['unit'].append(n_list[1])
        else:
            instart_fruits['unit'].append(item['unit'])
        price = item['pricing']['price'][1:]
        instart_fruits['price'].append(price)
    #for k in instart_fruits.keys():
    #   print(len(instart_fruits[k]))
    df = pd.DataFrame(instart_fruits)
    return df

def get_all(store_files,store):
    df = pd.DataFrame({'name':[],'store':[],'price':[],'unit':[],'image_url':[]})
    for i in range(len(store_files)):
        temp = load_json(store_files[i],store)
        df = pd.concat([df,temp]).reset_index(drop=True)
    return df

In [7]:
target_df = get_all(target_all,'Target1')
target_df = target_df[(target_df['unit']=='each') | 
                      (target_df['unit']=='bag') | (target_df['unit']=='lb')].reset_index(drop=True)
target_df = target_df.loc[3:,:]
target_df.loc[10,'unit'] = 'Bag'
target_df.loc[13,'unit'] = 'Bag'
target_df = target_df.drop([21]).reset_index(drop=True)
target_df.head()

,name,store,price,unit,image_url
0,Bluberries,Target1,3.49,each,https://d1s8987jlndkbs.cloudfront.net/assets/m...
1,Cantaloupe Melon,Target1,3.09,each,https://d2lnr5mha7bycj.cloudfront.net/product-...
2,Yellow Onion,Target1,0.89,each,https://d2lnr5mha7bycj.cloudfront.net/product-...
3,Pineapple,Target1,3.49,each,https://d2lnr5mha7bycj.cloudfront.net/product-...
4,Pink Lady (Cripps) Apple,Target1,1.19,each,https://d2lnr5mha7bycj.cloudfront.net/product-...


In [8]:
aldi_df = get_all(aldi_all,'ALDI1')
aldi_df.head()

,name,store,price,unit,image_url
0,Banana,ALDI1,0.49,each,https://d2lnr5mha7bycj.cloudfront.net/product-...
1,Strawberries,ALDI1,3.29,Package,https://d2lnr5mha7bycj.cloudfront.net/product-...
2,Blueberries,ALDI1,3.09,Package,https://d2lnr5mha7bycj.cloudfront.net/product-...
3,Organic Banana,ALDI1,0.59,each,https://d2lnr5mha7bycj.cloudfront.net/product-...
4,Large Avocado,ALDI1,0.79,each,https://d2lnr5mha7bycj.cloudfront.net/product-...


In [9]:
sch_df = get_all(sch_all,'Schnuks1')
sch_df = sch_df[(sch_df['unit']=='each') | (sch_df['unit']=='bag') | (sch_df['unit']=='lb')].reset_index(drop=True)
sch_df.head()

,name,store,price,unit,image_url
0,Banana,Schnuks1,0.46,each,https://d2lnr5mha7bycj.cloudfront.net/product-...
1,Strawberries,Schnuks1,4.19,each,https://d2lnr5mha7bycj.cloudfront.net/product-...
2,Organic Blueberries,Schnuks1,5.29,each,https://d2lnr5mha7bycj.cloudfront.net/product-...
3,Avocado,Schnuks1,0.84,each,https://d2lnr5mha7bycj.cloudfront.net/product-...
4,Halos California Clementines,Schnuks1,3.99,each,https://d2lnr5mha7bycj.cloudfront.net/product-...


In [10]:
gfs_df = get_all(gfs_all,'GFS')
gfs_df.loc[21,'name'] = 'Bauman Orchards Inc. Gala Fresh Apples'
gfs_df.loc[21,'unit'] = 'Bag'
gfs_df.head()

,name,store,price,unit,image_url
0,SK From Sunkist Lemons,GFS,4.79,each,https://d2lnr5mha7bycj.cloudfront.net/product-...
1,Granny Smith Apples,GFS,5.79,Bag,https://d2lnr5mha7bycj.cloudfront.net/product-...
2,Cored Pineapple,GFS,4.79,each,https://d2lnr5mha7bycj.cloudfront.net/product-...
3,Apple Golden Delicious,GFS,4.99,each,https://d2lnr5mha7bycj.cloudfront.net/product-...
4,Sunbelle GreenBelle Blackberries,GFS,3.99,each,https://d2lnr5mha7bycj.cloudfront.net/product-...


## Walmart

In [11]:
def load_walmart(file_path):
    with open(file_path) as f:
        json_data = json.load(f)
    walmart_fruits = {'name':[],'store':[], 'price':[], 'unit':[],'image_url':[]}
    walmart_fruits['store'] = 'Walmart1'
    walmart_fruits['price'] = 0
    for item in json_data['items']:
        walmart_fruits['image_url'].append(item['imageProps']['src'])
        n_list = item['title'].split(',')
        walmart_fruits['name'].append(n_list[0])
        if len(n_list) > 1:
            walmart_fruits['unit'].append(n_list[1])
        else:
            walmart_fruits['unit'].append('each')
    df = pd.DataFrame(walmart_fruits)      
    return df

walmart_all = root+'walmart.json'
walmart_df = load_walmart(walmart_all)
walmart_df.head()

,name,store,price,unit,image_url
0,Bananas,Walmart1,0,each,https://i5.walmartimages.com/asr/209bb8a0-30ab...
1,Hass Avocados,Walmart1,0,each,https://i5.walmartimages.com/asr/098962b7-7d67...
2,Fresh Red Seedless Grapes,Walmart1,0,per lb,https://i5.walmartimages.com/asr/da7810bf-fd66...
3,Honeycrisp Apples,Walmart1,0,each,https://i5.walmartimages.com/asr/fdffb720-8b75...
4,Seedless Watermelon,Walmart1,0,Each,https://i5.walmartimages.com/asr/7eaaef1a-e70d...


## Generate Products.csv

In [12]:
temp = pd.concat([aldi_df,gfs_df]).reset_index(drop=True)
temp = pd.concat([temp,meijer_df]).reset_index(drop=True)
temp = pd.concat([temp,sch_df]).reset_index(drop=True)
temp = pd.concat([temp,target_df]).reset_index(drop=True)
fruits_all = pd.concat([temp,walmart_df]).reset_index(drop=True)

In [13]:
fruits_all.reset_index(inplace=True)
fruits_all = fruits_all.rename(columns = {'index':'ProductID','name':'ProductName','store':'StoreID',
                                          'price':'UnitPrice','unit':'Unit','image_url':'ImageURL'})
fruits_all['Availability'] = 'Y'

In [14]:
fruits_all

,ProductID,ProductName,StoreID,UnitPrice,Unit,ImageURL,Availability
0,0,Banana,ALDI1,0.49,each,https://d2lnr5mha7bycj.cloudfront.net/product-...,Y
1,1,Strawberries,ALDI1,3.29,Package,https://d2lnr5mha7bycj.cloudfront.net/product-...,Y
2,2,Blueberries,ALDI1,3.09,Package,https://d2lnr5mha7bycj.cloudfront.net/product-...,Y
3,3,Organic Banana,ALDI1,0.59,each,https://d2lnr5mha7bycj.cloudfront.net/product-...,Y
4,4,Large Avocado,ALDI1,0.79,each,https://d2lnr5mha7bycj.cloudfront.net/product-...,Y
...,...,...,...,...,...,...,...
272,272,Clementines,Walmart1,0,3 lbs,https://i5.walmartimages.com/asr/b1748a2b-6cb1...,Y
273,273,Organic 1lb strawberries,Walmart1,0,each,https://i5.walmartimages.com/asr/e31e0ab4-d9bc...,Y
274,274,New York Apple Sales Cortland Apples,Walmart1,0,5 lbs,https://i5.walmartimages.com/asr/edfccb90-888a...,Y
275,275,Fresh Blueberries,Walmart1,0,6 oz,https://i5.walmartimages.com/asr/08ec1546-6ced...,Y


In [ ]:
fruits_all.to_csv('products.csv',index=False,header =False)